# 1.5 "파인 튜닝"으로 정확도 향상을 실현하는 방법

- 이 파일에서는 학습된 VGG 모델을 사용하여 파인 튜닝을 통해 개미와 벌 이미지를 분류하는 모델을 학습합니다.


# 학습 목표

1. PyTorch에서 GPU를 사용하는 구현 코드를 작성할 수 있게 된다.
2. 최적화 기법 설정에서 각 층별로 다른 학습률을 설정한 파인 튜닝을 구현할 수 있게 된다.
3. 학습한 네트워크를 저장하고 로드할 수 있게 된다.


# 사전 준비

- 1.4절에서 설명한 AWS EC2의 GPU 인스턴스를 사용합니다.
- ~~ 그러나 나는 colab이 있다 ~~

In [4]:
# 패키지 import
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

import sys

In [5]:
# 랜덤 시드 생성
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# Dataset과 DataLoader 생성

In [7]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/pytorch_advanced/1_image_classification')

# util 폴더의 dataloader_image_classification.py 사용
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# 파일 경로 리스트 설정
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Dataset 작성
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')


# DataLoader 작성
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# dictionary에 DataLoader 저장
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/pytorch_advanced/1_image_classification/data/hymenoptera_data/train/**/*.jpg
/content/drive/MyDrive/pytorch_advanced/1_image_classification/data/hymenoptera_data/val/**/*.jpg


# 네트워크 모델 작성


In [8]:
# 학습된 VGG-16 모델 로드

# VGG-16 인스턴스 생성
use_pretrained = True  # 학습된 파라미터 사용
net = models.vgg16(pretrained=use_pretrained)

# VGG16의 마지막 출력층을 2로 설정 (개미 or 벌)
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 훈련 모드로 설정
net.train()

print('모델 설정 완료')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 97.6MB/s]


모델 설정 완료


# 손실함수 정의


In [9]:
# 손실함수 설정
criterion = nn.CrossEntropyLoss()

# 최적화 기법 설정

In [10]:
# 파인튜닝으로 학습할 파라미터들을 리스트에 저장

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습시킬 층의 파라미터 이름 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 파라미터를 각 리스트에 저장
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2에 저장：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3에 저장：", name)

    else:
        param.requires_grad = False
        print("학습 안 함：", name)


params_to_update_1에 저장： features.0.weight
params_to_update_1에 저장： features.0.bias
params_to_update_1에 저장： features.2.weight
params_to_update_1에 저장： features.2.bias
params_to_update_1에 저장： features.5.weight
params_to_update_1에 저장： features.5.bias
params_to_update_1에 저장： features.7.weight
params_to_update_1에 저장： features.7.bias
params_to_update_1에 저장： features.10.weight
params_to_update_1에 저장： features.10.bias
params_to_update_1에 저장： features.12.weight
params_to_update_1에 저장： features.12.bias
params_to_update_1에 저장： features.14.weight
params_to_update_1에 저장： features.14.bias
params_to_update_1에 저장： features.17.weight
params_to_update_1에 저장： features.17.bias
params_to_update_1에 저장： features.19.weight
params_to_update_1에 저장： features.19.bias
params_to_update_1에 저장： features.21.weight
params_to_update_1에 저장： features.21.bias
params_to_update_1에 저장： features.24.weight
params_to_update_1에 저장： features.24.bias
params_to_update_1에 저장： features.26.weight
params_to_update_1에 저장： features.26.bias


In [11]:
# 최적화 방법 설정
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)


# 학습 및 검증 실시

In [12]:
# 모델 학습 함수

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 초기 설정
    # GPU를 사용할 수 있는지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치:", device)

    # 모델을 GPU로 이동
    net.to(device)

    # 입력데이터 크기가 고정되어 있으면 속도 향상
    torch.backends.cudnn.benchmark = True

    # epoch 루프
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # 각 epoch의 학습 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 학습 모드로 설정
            else:
                net.eval()   # 모델을 검증 모드로 설정

            epoch_loss = 0.0  # 각 epoch의 손실 합계
            epoch_corrects = 0  # 각 epoch의 정답 수

            # 미학습 상태에서의 검증 성능을 확인하기 위해, epoch=0의 학습은 생략
            if (epoch == 0) and (phase == 'train'):
                continue

            # 데이터로더에서 미니 배치를 가져오는 루프
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPU를 사용할 수 있으면 데이터를 GPU로 보냄
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 옵티마이저 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 손실 계산
                    _, preds = torch.max(outputs, 1)  # 라벨 예측

                    # 학습 시 역전파 수행
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    epoch_loss += loss.item() * inputs.size(0)  # 손실 합계 업데이트
                    # 정답 수 합계 업데이트
                    epoch_corrects += torch.sum(preds == labels.data)

            # 각 epoch의 손실과 정확도 출력
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


In [13]:
# 학습 및 검증
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


사용 장치: cpu
Epoch 1/2
-------------


100%|██████████| 5/5 [02:37<00:00, 31.52s/it]


val Loss: 0.7702 Acc: 0.4444
Epoch 2/2
-------------


100%|██████████| 8/8 [10:20<00:00, 77.51s/it]


train Loss: 0.5254 Acc: 0.6996


100%|██████████| 5/5 [01:47<00:00, 21.47s/it]

val Loss: 0.1737 Acc: 0.9608


# 학습된 모델 저장 및 로드

In [14]:
# PyTorch 모델 파라미터 저장
save_path = '/content/drive/MyDrive/pytorch_advanced/1_image_classification/saved_models/weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)


In [16]:
# PyTorch 모델 파라미터 로드
load_path = '/content/drive/MyDrive/pytorch_advanced/1_image_classification/saved_models/weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU -> CPU
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)


<ipython-input-16-8b35e65b1050>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_weights = torch.load(load_path)
<ipython-input-16-8b35e65b1050>:7: FutureWarning: You a

<All keys matched successfully>

以上